In [1]:
import pandas as pd
import numpy as np
import os
import sys
import polars as pl
import json
from joblib import Parallel, delayed
import deepchem
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Configure Polars 
cfg = pl.Config()
cfg.set_tbl_rows(20)
cfg.set_tbl_cols(50)
from sklearn.model_selection import StratifiedGroupKFold, GroupKFold, KFold, GroupShuffleSplit, ShuffleSplit, StratifiedKFold
import gc
import random

/tmp/ipykernel_27103/3549581206.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepch

In [2]:
PERCENTILES = [0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.975, 0.99]

In [3]:
with open('/home/dangnh36/datasets/competitions/leash_belka/processed/meta/building_blocks.json', 'r') as f:
    bbs_meta = json.load(f)

[print(k, '-->', len(v)) for k, v in bbs_meta.items()]

train_bbs --> 1145
train_bb1s --> 271
train_bb2s --> 693
train_bb3s --> 872
test_bb1s --> 341
test_bb2s --> 1140
test_bb3s --> 1389
test_bbs --> 2110
all_bbs --> 2110


[None, None, None, None, None, None, None, None, None]

In [4]:
with open('/home/dangnh36/datasets/competitions/leash_belka/processed/meta/scaffolds.json', 'r') as f:
    scaffolds = json.load(f)
print(len(scaffolds))

5971685


In [5]:
train_scaffolds = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/train_scaffold.csv').collect()
train_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/train_v2.csv')\
    .select(
        pl.col('molecule'),
        pl.col('bb1', 'bb2', 'bb3').cast(pl.UInt16),
        pl.col('BRD4', 'HSA', 'sEH').cast(pl.UInt8),
        scaffold_idx = train_scaffolds['scaffold_idx'],
    )\
    .collect()
print(train_df.estimated_size('mb'))
train_df

8601.91998577118


molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx
str,u16,u16,u16,u8,u8,u8,i64
"""C#CCOc1ccc(CNc…",1640,1653,765,0,0,0,4283326
"""C#CCOc1ccc(CNc…",1640,1653,205,0,0,0,4486646
"""C#CCOc1ccc(CNc…",1640,1653,1653,0,0,0,1015728
"""C#CCOc1ccc(CNc…",1640,1653,146,0,0,0,5301385
"""C#CCOc1ccc(CNc…",1640,1653,439,0,0,0,5301385
"""C#CCOc1ccc(CNc…",1640,1653,196,0,0,0,5301385
"""C#CCOc1ccc(CNc…",1640,1653,253,0,0,0,5301385
"""C#CCOc1ccc(CNc…",1640,1653,1219,0,0,0,5301385
"""C#CCOc1ccc(CNc…",1640,1653,604,0,0,0,543172


In [6]:
test_scaffolds = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/test_scaffold.csv').collect()
test_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/test_v2.csv')\
    .select(
        pl.col('id','molecule'),
        pl.col('bb1', 'bb2', 'bb3').cast(pl.UInt16),
        pl.col('protein'),
        scaffold_idx = test_scaffolds['scaffold_idx']
    ).group_by('molecule').first().collect()
print(test_df.estimated_size('mb'))
test_df

84.21934127807617


molecule,id,bb1,bb2,bb3,protein,scaffold_idx
str,i64,u16,u16,u16,str,i64
"""O=C(N[Dy])[C@H…",295506661,1040,1105,2027,"""HSA""",386860
"""Cc1cc(C)n(-c2c…",295556866,1555,870,1501,"""sEH""",2800116
"""CSc1cccc(-c2cc…",296102494,2044,750,865,"""sEH""",391264
"""Cc1c(C(=O)N2C[…",295495248,1040,355,1866,"""HSA""",5300353
"""COc1ccc(C(=O)N…",295669456,962,2019,135,"""BRD4""",3162315
"""Cc1ccc(O)c(CNc…",295872682,1945,25,1792,"""BRD4""",4490305
"""O=C(N[Dy])[C@H…",295912171,1493,1980,44,"""BRD4""",41190
"""COC(CNc1nc(NCC…",296556446,608,1400,1787,"""BRD4""",1784398
"""CC1CCC(C(=O)N2…",296765451,1287,106,323,"""BRD4""",4892798


In [7]:
df = pl.concat([train_df.select(pl.col('molecule', 'bb1', 'bb2', 'bb3', 'scaffold_idx'), ori=pl.lit(0)),
                test_df.select(pl.col('molecule', 'bb1', 'bb2', 'bb3', 'scaffold_idx'), ori= pl.lit(1))])
df = df.with_columns(
    pl.col('molecule').count().over('scaffold_idx').alias('mol_per_scaf')
)
df

molecule,bb1,bb2,bb3,scaffold_idx,ori,mol_per_scaf
str,u16,u16,u16,i64,i32,u32
"""C#CCOc1ccc(CNc…",1640,1653,765,4283326,0,844
"""C#CCOc1ccc(CNc…",1640,1653,205,4486646,0,9444
"""C#CCOc1ccc(CNc…",1640,1653,1653,1015728,0,9030
"""C#CCOc1ccc(CNc…",1640,1653,146,5301385,0,26814
"""C#CCOc1ccc(CNc…",1640,1653,439,5301385,0,26814
"""C#CCOc1ccc(CNc…",1640,1653,196,5301385,0,26814
"""C#CCOc1ccc(CNc…",1640,1653,253,5301385,0,26814
"""C#CCOc1ccc(CNc…",1640,1653,1219,5301385,0,26814
"""C#CCOc1ccc(CNc…",1640,1653,604,543172,0,844


In [10]:
(df.filter(pl.col('ori') == 0)['molecule'] == train_df['molecule']).all()

True

In [11]:
train_df = train_df.with_columns(mol_per_scaf = df.filter(pl.col('ori') == 0)['mol_per_scaf'])
train_df

molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf
str,u16,u16,u16,u8,u8,u8,i64,u32
"""C#CCOc1ccc(CNc…",1640,1653,765,0,0,0,4283326,844
"""C#CCOc1ccc(CNc…",1640,1653,205,0,0,0,4486646,9444
"""C#CCOc1ccc(CNc…",1640,1653,1653,0,0,0,1015728,9030
"""C#CCOc1ccc(CNc…",1640,1653,146,0,0,0,5301385,26814
"""C#CCOc1ccc(CNc…",1640,1653,439,0,0,0,5301385,26814
"""C#CCOc1ccc(CNc…",1640,1653,196,0,0,0,5301385,26814
"""C#CCOc1ccc(CNc…",1640,1653,253,0,0,0,5301385,26814
"""C#CCOc1ccc(CNc…",1640,1653,1219,0,0,0,5301385,26814
"""C#CCOc1ccc(CNc…",1640,1653,604,0,0,0,543172,844


In [12]:
train_df = train_df.with_columns((pl.col('BRD4') * 4 + pl.col('HSA') * 2 + pl.col('sEH')).alias('label'))
train_df.select(pl.n_unique('*'))

molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
98415610,271,693,872,2,2,2,5711873,2388,8


In [13]:
del test_df, train_scaffolds, test_scaffolds, df
gc.collect()

47

In [15]:
def train_test_split(df, test_only_bb_idxs, train_only_bb_idxs = None, random_state = 42):

    # 369_039 is estimated number of overlapped-BB samples for real test set (the remaining 508_983 is completely non-shared)
    stage23_factor = 369_039 / 508_983
    
    # 158_699 is estimated number of samples from Stage 2 (Scaffold split) for real test set
    stage2_factor = 158_699 / 508_983
    
    all_bbs = bbs_meta['all_bbs']
    # subset = 255: UNK
    df = df.with_columns(subset = pl.lit(255).cast(pl.UInt8))

    # Stage 1: BB-split
    # in this stage, we hold out a set of test-only building blocks
    print('##### STAGE 1 #####')
    print('Number of test-only building blocks:', len(test_only_bb_idxs))

    # For pre-defined competition train-test split, & or | is both equivalent
    # But for user-defined custom train-test split on train data, it really matters
    if train_only_bb_idxs is None:
        stage2_df = df.filter(~(pl.col('bb1').is_in(test_only_bb_idxs) | pl.col('bb2').is_in(test_only_bb_idxs) | pl.col('bb3').is_in(test_only_bb_idxs)))
    else:
        stage2_df = df.filter((pl.col('bb1').is_in(train_only_bb_idxs) & pl.col('bb2').is_in(train_only_bb_idxs) & pl.col('bb3').is_in(train_only_bb_idxs)))
    test1_df = df.filter(
        (pl.col('bb1').is_in(test_only_bb_idxs) | pl.col('bb2').is_in(test_only_bb_idxs) | pl.col('bb3').is_in(test_only_bb_idxs))
    )
    # subset = 0: non-share, i.e NONE of the BB is shared between train and test
    # subset = 1: at-least one BB is not shared, e.g A-B-C where A and C is shared, but B is not shared
    test1_df = test1_df.with_columns(
        pl.when((pl.col('bb1').is_in(test_only_bb_idxs) & pl.col('bb2').is_in(test_only_bb_idxs) & pl.col('bb3').is_in(test_only_bb_idxs)))\
        .then(0).otherwise(1).alias('subset'))

    # number of non-shared molecules
    test1_non_shared_df = test1_df.filter(pl.col('subset') == 0)
    num_non_shared = test1_non_shared_df.shape[0]
    stage2_test_size = round(stage2_factor * num_non_shared)
    stage23_test_size = round(stage23_factor * num_non_shared)
    print('Number of non-shared molecules:', test1_non_shared_df.shape)
    print(f'Stage2={stage2_test_size}, Stage3={stage23_test_size}')

    print('Stage 1 Test set statistic:')
    display(test1_df.select(pl.n_unique('*')))
    display(test1_df.describe(percentiles = PERCENTILES))
    print('Stage 1 NON-SHARED Test set statistic:')
    display(test1_non_shared_df.select(pl.n_unique('*')))
    display(test1_non_shared_df.describe(percentiles = PERCENTILES))
    print('Stage 1 Train set statistic:')
    display(stage2_df.select(pl.n_unique('*')))
    display(stage2_df.describe(percentiles = PERCENTILES))
    gc.collect()

    # Stage 2
    # Leave 20% molecules with most regular scaffolds: > 6116 mols/scaffold
    # Then do a Scaffold Split on the remaining 80% molecules
    # Scaffold Split is simply implemented with sklearn's GroupKFold
    print('##### STAGE 2 #####')
    stage2_df = stage2_df.with_columns(index = pl.Series(list(range(len(stage2_df)))))
    lower_stage2_df = stage2_df.filter(pl.col('mol_per_scaf').is_between(1, 6116))
    higher_stage2_df = stage2_df.filter((pl.col('mol_per_scaf') > 6116))
    if isinstance(stage2_test_size, int) and stage2_test_size > 1:
        stage2_test_pct = stage2_test_size / len(lower_stage2_df)
    else:
        raise ValueError
    stage2_n_folds = round(1 / stage2_test_pct)
    print('Number of folds:', stage2_n_folds)
    splitter = GroupKFold(n_splits=stage2_n_folds)
    # splitter = StratifiedGroupKFold(n_splits=stage2_n_folds, shuffle=True, random_state=random_state)
    for i, (tmp_train2_idxs, tmp_test2_idxs) in enumerate(splitter.split(range(len(lower_stage2_df)),
                                                                         lower_stage2_df['label'],
                                                                         lower_stage2_df['scaffold_idx'])):
        print(f"Fold {i}:")
        train2_idxs = lower_stage2_df[tmp_train2_idxs, 'index'].to_list()
        test2_idxs = lower_stage2_df[tmp_test2_idxs, 'index'].to_list()
        print('Fold train/test size:', len(train2_idxs), len(test2_idxs))
        break
    
    train2_df = stage2_df[train2_idxs]
    test2_df = stage2_df[test2_idxs].select(pl.col('*').exclude('index'))
    # subset = 2: scaffold split on lower-80% with overlapped BB (stage 2)
    test2_df = test2_df.with_columns(subset = pl.lit(2))
    stage3_df = pl.concat([train2_df, higher_stage2_df])

    print('Stage 2 Test set statistic:')
    display(test2_df.select(pl.n_unique('*')))
    display(test2_df.describe(percentiles = PERCENTILES))
    print('Stage 2 Train set statistic:')
    display(stage3_df.select(pl.n_unique('*')))
    display(stage3_df.describe(percentiles = PERCENTILES))
    gc.collect()

    # Stage 3: Random split on the remaining
    print('##### STAGE 3 #####')
    stage3_test_size = stage23_test_size - len(test2_df)
    if isinstance(stage3_test_size, int) and stage3_test_size > 1:
        stage3_test_pct = stage3_test_size / len(stage3_df)
    elif isinstance(stage3_test_size, float) and stage3_test_size < 1:
        stage3_test_pct = stage3_test_size
    else:
        raise ValueError

    stage3_n_folds = round(1 / stage3_test_pct)
    print('Number of folds:', stage3_n_folds, stage3_test_size)
    # splitter = KFold(n_splits=stage3_n_folds, shuffle=True, random_state=random_state)
    splitter = StratifiedKFold(n_splits=stage3_n_folds, shuffle=True, random_state=random_state)
    for i, (train3_idxs, test3_idxs) in enumerate(splitter.split(list(range(len(stage3_df))), stage3_df['label'])):
        print(f"Fold {i}:")
        print('Fold train/test size:', len(train3_idxs), len(test3_idxs))
        break
    train3_df = stage3_df[train3_idxs]
    test3_df = stage3_df[test3_idxs].select(pl.col('*').exclude('index'))
    # subset = 3: result of Stage 3: Random Split on the remaining
    test3_df = test3_df.with_columns(subset = pl.lit(3))
    
    print('Stage 3 Test set statistic:')
    display(test3_df.select(pl.n_unique('*')))
    display(test3_df.describe(percentiles = PERCENTILES))
    print('Stage 3 Train set statistic:')
    display(train3_df.select(pl.n_unique('*')))
    display(train3_df.describe(percentiles = PERCENTILES))
    gc.collect()

    test23_df = pl.concat([test2_df, test3_df])
    print('Test 2+3 set statistic:')
    display(test23_df.select(pl.n_unique('*')))
    display(test23_df.describe(percentiles = PERCENTILES))
    
    train_df = train3_df
    test_df = pl.concat([test1_df, test2_df, test3_df])

    print('Final Test set statistic:')
    display(test_df.select(pl.n_unique('*')))
    display(test_df.describe(percentiles = PERCENTILES))
    print('Final Train set statistic:')
    display(train_df.select(pl.n_unique('*')))
    display(train_df.describe(percentiles = PERCENTILES))
    gc.collect()

    train_df = train_df.select(pl.col('*').exclude('subset'))
    return train_df, test_df

In [16]:
BB1_SPLIT = 8
BB23_SPLIT = 8

with open(f'/home/dangnh36/datasets/competitions/leash_belka/processed/meta/bb_split_{BB1_SPLIT}_{BB23_SPLIT}.json', 'r') as f:
    global_fold_meta = json.load(f)

In [17]:
BB1_FOLD = str(4)
BB23_FOLD = str(6)
test_only_bb1_idxs = global_fold_meta[BB1_FOLD]['bb1']['test']
test_only_bb23_idxs = global_fold_meta[BB1_FOLD]['bb23'][BB23_FOLD]['test']
train_only_bb1_idxs = global_fold_meta[BB1_FOLD]['bb1']['train']
train_only_bb23_idxs = global_fold_meta[BB1_FOLD]['bb23'][BB23_FOLD]['train']


assert len(set(test_only_bb1_idxs).intersection(set(test_only_bb23_idxs))) == 0
assert len(set(train_only_bb1_idxs).intersection(set(train_only_bb23_idxs))) == 0

train_only_bb_idxs = set(train_only_bb1_idxs).union(set(train_only_bb23_idxs))
test_only_bb_idxs = set(test_only_bb1_idxs).union(set(test_only_bb23_idxs))

assert len(set(train_only_bb_idxs).intersection(set(test_only_bb_idxs))) == 0
assert len(train_only_bb_idxs.union(test_only_bb_idxs)) == len(bbs_meta['train_bbs'])

len(test_only_bb1_idxs), len(test_only_bb23_idxs), len(test_only_bb_idxs)

(33, 103, 136)

In [18]:
train_df = train_df.with_columns(ori_index = pl.Series(list(range(len(train_df)))))
train_df

molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label,ori_index
str,u16,u16,u16,u8,u8,u8,i64,u32,u8,i64
"""C#CCOc1ccc(CNc…",1640,1653,765,0,0,0,4283326,844,0,0
"""C#CCOc1ccc(CNc…",1640,1653,205,0,0,0,4486646,9444,0,1
"""C#CCOc1ccc(CNc…",1640,1653,1653,0,0,0,1015728,9030,0,2
"""C#CCOc1ccc(CNc…",1640,1653,146,0,0,0,5301385,26814,0,3
"""C#CCOc1ccc(CNc…",1640,1653,439,0,0,0,5301385,26814,0,4
"""C#CCOc1ccc(CNc…",1640,1653,196,0,0,0,5301385,26814,0,5
"""C#CCOc1ccc(CNc…",1640,1653,253,0,0,0,5301385,26814,0,6
"""C#CCOc1ccc(CNc…",1640,1653,1219,0,0,0,5301385,26814,0,7
"""C#CCOc1ccc(CNc…",1640,1653,604,0,0,0,543172,844,0,8


In [19]:
final_train_df, final_test_df = train_test_split(train_df, test_only_bb_idxs, train_only_bb_idxs = None, random_state = 42)

##### STAGE 1 #####
Number of test-only building blocks: 136
Number of non-shared molecules: (173831, 12)
Stage2=54200, Stage3=126036
Stage 1 Test set statistic:


molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label,ori_index,subset
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
31579445,271,693,872,2,2,2,2548438,2388,8,31579445,2


statistic,molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label,ori_index,subset
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""31579445""",3.1579445e7,3.1579445e7,3.1579445e7,3.1579445e7,3.1579445e7,3.1579445e7,3.1579445e7,3.1579445e7,3.1579445e7,3.1579445e7,3.1579445e7
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,1119.771436,1069.935063,1070.24005,0.004371,0.003866,0.006899,3.0424e6,18437.700256,0.032116,4.8620e7,0.994495
"""std""",null,587.45225,616.061992,613.196835,0.06597,0.06206,0.082771,1.7261e6,65057.518335,0.306994,2.7679e7,0.073988
"""min""","""C#CCOc1ccc(CNc…",7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"""1%""",null,36.0,25.0,20.0,0.0,0.0,0.0,68630.0,1.0,0.0,315794.0,1.0
"""2.5%""",null,76.0,49.0,46.0,0.0,0.0,0.0,158172.0,2.0,0.0,2.248088e6,1.0
"""5%""",null,171.0,110.0,113.0,0.0,0.0,0.0,289734.0,3.0,0.0,5.509581e6,1.0
"""10%""",null,346.0,154.0,206.0,0.0,0.0,0.0,614327.0,9.0,0.0,1.0203882e7,1.0


Stage 1 NON-SHARED Test set statistic:


molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label,ori_index,subset
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
173831,33,92,103,2,2,2,29933,1460,6,173831,1


...

molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label,ori_index,subset
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
31705451,271,693,872,2,2,2,2560137,2388,8,31705451,4


statistic,molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label,ori_index,subset
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""31705451""",3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,1119.568927,1070.092052,1070.09145,0.004372,0.003869,0.006901,3.0424e6,18398.32598,0.032127,4.8625e7,1.000756
"""std""",null,587.48397,616.031136,613.155552,0.065976,0.062085,0.082785,1.7257e6,64986.265685,0.307038,2.7684e7,0.127479
"""min""","""C#CCOc1ccc(CNc…",7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"""1%""",null,36.0,25.0,20.0,0.0,0.0,0.0,68587.0,1.0,0.0,317055.0,1.0
"""2.5%""",null,76.0,49.0,45.0,0.0,0.0,0.0,158224.0,2.0,0.0,2.248638e6,1.0
"""5%""",null,171.0,110.0,113.0,0.0,0.0,0.0,290045.0,3.0,0.0,5.508469e6,1.0
"""10%""",null,346.0,154.0,206.0,0.0,0.0,0.0,614667.0,9.0,0.0,1.02036e7,1.0


Final Train set statistic:


molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label,ori_index,subset,index
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
66710159,238,601,769,2,2,2,4359915,2387,8,66710159,1,66710159


statistic,molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label,ori_index,subset,index
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""66710159""",6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,1074.413092,1079.332427,1033.021232,0.004772,0.004283,0.007581,3.0227e6,14104.587721,0.035236,4.9485e7,255.0,3.3417e7
"""std""",null,591.517029,609.725897,610.571276,0.068916,0.065305,0.086739,1.7249e6,55811.110771,0.321424,2.8744e7,0.0,1.9294e7
"""min""","""C#CCOc1ccc(CNc…",7.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,363110.0,255.0,0.0
"""1%""",null,21.0,26.0,15.0,0.0,0.0,0.0,63750.0,1.0,0.0,1.227679e6,255.0,668354.0
"""2.5%""",null,76.0,45.0,44.0,0.0,0.0,0.0,155462.0,1.0,0.0,2.523782e6,255.0,1.670883e6
"""5%""",null,141.0,90.0,98.0,0.0,0.0,0.0,286317.0,3.0,0.0,4.686871e6,255.0,3.341846e6
"""10%""",null,273.0,158.0,205.0,0.0,0.0,0.0,606564.0,8.0,0.0,9.734228e6,255.0,6.683334e6


In [20]:
final_test_df.shape, final_train_df.shape

((31705451, 12), (66710159, 12))

In [21]:
display(final_test_df.select(pl.col('BRD4', 'HSA', 'sEH').mean() * 100))
display(final_test_df.group_by('subset').count().sort('subset'))
display(final_test_df.select(pl.n_unique('*')))
display(final_test_df.describe(percentiles=PERCENTILES))

BRD4,HSA,sEH
f64,f64,f64
0.43719,0.386946,0.690096


subset,count
i32,u32
0,173831
1,31405614
2,54197
3,71809


molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label,ori_index,subset
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
31705451,271,693,872,2,2,2,2560137,2388,8,31705451,4


statistic,molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label,ori_index,subset
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""31705451""",3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7,3.1705451e7
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,1119.568927,1070.092052,1070.09145,0.004372,0.003869,0.006901,3.0424e6,18398.32598,0.032127,4.8625e7,1.000756
"""std""",null,587.48397,616.031136,613.155552,0.065976,0.062085,0.082785,1.7257e6,64986.265685,0.307038,2.7684e7,0.127479
"""min""","""C#CCOc1ccc(CNc…",7.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
"""1%""",null,36.0,25.0,20.0,0.0,0.0,0.0,68587.0,1.0,0.0,317055.0,1.0
"""2.5%""",null,76.0,49.0,45.0,0.0,0.0,0.0,158224.0,2.0,0.0,2.248638e6,1.0
"""5%""",null,171.0,110.0,113.0,0.0,0.0,0.0,290045.0,3.0,0.0,5.508469e6,1.0
"""10%""",null,346.0,154.0,206.0,0.0,0.0,0.0,614667.0,9.0,0.0,1.02036e7,1.0


In [22]:
display(final_train_df.select(pl.col('BRD4', 'HSA', 'sEH').mean() * 100))
display(final_train_df.select(pl.n_unique('*')))
display(final_train_df.describe(percentiles=PERCENTILES))
display()

BRD4,HSA,sEH
f64,f64,f64
0.477215,0.428311,0.758106


molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label,ori_index,index
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
66710159,238,601,769,2,2,2,4359915,2387,8,66710159,66710159


statistic,molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label,ori_index,index
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""66710159""",6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7,6.6710159e7
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,1074.413092,1079.332427,1033.021232,0.004772,0.004283,0.007581,3.0227e6,14104.587721,0.035236,4.9485e7,3.3417e7
"""std""",null,591.517029,609.725897,610.571276,0.068916,0.065305,0.086739,1.7249e6,55811.110771,0.321424,2.8744e7,1.9294e7
"""min""","""C#CCOc1ccc(CNc…",7.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,363110.0,0.0
"""1%""",null,21.0,26.0,15.0,0.0,0.0,0.0,63750.0,1.0,0.0,1.227679e6,668354.0
"""2.5%""",null,76.0,45.0,44.0,0.0,0.0,0.0,155462.0,1.0,0.0,2.523782e6,1.670883e6
"""5%""",null,141.0,90.0,98.0,0.0,0.0,0.0,286317.0,3.0,0.0,4.686871e6,3.341846e6
"""10%""",null,273.0,158.0,205.0,0.0,0.0,0.0,606564.0,8.0,0.0,9.734228e6,6.683334e6


# Save

In [23]:
final_train_df

molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label,ori_index,index
str,u16,u16,u16,u8,u8,u8,i64,u32,u8,i64,i64
"""C#CCOc1ccc(CNc…",982,1653,765,0,0,0,4283326,844,0,363110,0
"""C#CCOc1ccc(CNc…",982,1653,604,0,0,0,543172,844,0,363118,7
"""C#CCOc1ccc(CNc…",982,1653,121,0,0,0,2571428,864,0,363119,8
"""C#CCOc1ccc(CNc…",982,1653,1134,0,0,0,4654763,1728,0,363122,10
"""C#CCOc1ccc(CNc…",982,1653,1896,0,0,0,5600610,864,0,363123,11
"""C#CCOc1ccc(CNc…",982,1653,1617,0,0,0,2301320,844,0,363127,15
"""C#CCOc1ccc(CNc…",982,1653,1150,0,0,0,4484748,2595,0,363129,17
"""C#CCOc1ccc(CNc…",982,1653,802,0,0,0,1825155,844,0,363131,19
"""C#CCOc1ccc(CNc…",982,1653,815,0,0,0,2021587,3396,0,363133,20


In [24]:
save_train_df = final_train_df.select(
    pl.col('ori_index').alias('index'),
    pl.col('label')
)
save_train_df

index,label
i64,u8
363110,0
363118,0
363119,0
363122,0
363123,0
363127,0
363129,0
363131,0
363133,0


In [25]:
final_test_df

molecule,bb1,bb2,bb3,BRD4,HSA,sEH,scaffold_idx,mol_per_scaf,label,ori_index,subset
str,u16,u16,u16,u8,u8,u8,i64,u32,u8,i64,i32
"""C#CCOc1ccc(CNc…",1640,1653,765,0,0,0,4283326,844,0,0,1
"""C#CCOc1ccc(CNc…",1640,1653,205,0,0,0,4486646,9444,0,1,1
"""C#CCOc1ccc(CNc…",1640,1653,1653,0,0,0,1015728,9030,0,2,1
"""C#CCOc1ccc(CNc…",1640,1653,146,0,0,0,5301385,26814,0,3,1
"""C#CCOc1ccc(CNc…",1640,1653,439,0,0,0,5301385,26814,0,4,1
"""C#CCOc1ccc(CNc…",1640,1653,196,0,0,0,5301385,26814,0,5,1
"""C#CCOc1ccc(CNc…",1640,1653,253,0,0,0,5301385,26814,0,6,1
"""C#CCOc1ccc(CNc…",1640,1653,1219,0,0,0,5301385,26814,0,7,1
"""C#CCOc1ccc(CNc…",1640,1653,604,0,0,0,543172,844,0,8,1


In [26]:
save_test_df = final_test_df.select(
    pl.col('ori_index').alias('index'),
    pl.col('label'),
    pl.col('subset')
)
save_test_df

index,label,subset
i64,u8,i32
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
5,0,1
6,0,1
7,0,1
8,0,1


In [27]:
save_train_df.write_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/cv/v1/train.csv')
save_test_df.write_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/cv/v1/val.csv')